In [ ]:
!pip install pymongo
!pip install SQLAlchemy

from pymongo import MongoClient
from datetime import datetime
import pandas as pd
from sqlalchemy import create_engine, inspect
from dateutil.relativedelta import relativedelta

HOST = "34.155.6.23"
MONGO_PORT = 27017
PSQL_PORT = 5432
DB_USER = "pacha"
DB_PASSWORD = "passer"

In [71]:
# --- Connexion à MongoDB  ---
client = MongoClient(f"mongodb://{HOST}:{MONGO_PORT}/") 
db = client["videoGames"]
collection = db["games"]

cursor = collection.find()
df = pd.DataFrame(list(cursor))
print(df.head(2))


                        _id  overall  verified   reviewTime      reviewerID  \
0  68cbd3532d9228a925e9adee      4.0     False  07 27, 2015  A1JGAP0185YJI6   
1  68cbd3532d9228a925e9adef      5.0      True  10 17, 2015  A1HP7NVNPFMA4N   

         asin reviewerName                                         reviewText  \
0  0700026657       travis  I played it a while but it was alright. The st...   
1  0700026657  Ambrosia075  This game is a bit hard to get the hang of, bu...   

                                       summary  unixReviewTime vote style  \
0  But in spite of that it was fun, I liked it      1437955200  NaN   NaN   
1                  but when you do it's great.      1445040000  NaN   NaN   

  image  
0   NaN  
1   NaN  


In [ ]:
# --- Aggrégation ---
date = datetime.fromisoformat("2015-01-01")
six_months_ago = date - relativedelta(months=6)
df["review_date"] = pd.to_datetime(df["reviewTime"], format="%m %d, %Y")
df_last_6_months = df[(df["review_date"] >= six_months_ago) & (df["review_date"] <= date)]
df_agg = df_last_6_months.groupby("asin").agg(
    average_rating=("overall", "mean"),
    rating_count=("reviewerID", "count"),
    oldest_rating=("review_date", "min"),
    latest_rating=("review_date", "max")
)

df_agg = df_agg.rename_axis("game_id").reset_index()

# --- Insertion dans la DB ---
engine = create_engine(f"postgresql://{DB_USER}:{DB_PASSWORD}@{HOST}:{PSQL_PORT}/video_games")
df_agg.to_sql("top_rated_games", engine, if_exists="append", index=False)



715

In [64]:
# --- Vérification des données dans la table ---
inspector = inspect(engine)
tables = inspector.get_table_names()
print("Tables dans la DB :", tables)

df_games = pd.read_sql("SELECT * FROM top_rated_games;", engine)  # Remplace "games" par le nom de ta table
print(df_games.head())

Tables dans la DB : ['top_rated_games']
      game_id  average_rating  rating_count oldest_rating latest_rating
0  0700026398        4.000000             4    2014-07-01    2014-08-12
1  0700026657        3.333333             6    2014-08-02    2014-12-25
2  0804161380        5.000000            20    2014-07-05    2014-12-24
3  3828770193        5.000000             2    2014-10-18    2014-10-18
4  6050036071        4.750000             8    2014-10-02    2014-12-14
